In [40]:
import aiohttp
import asyncio

In [41]:
import os
import re
import requests
import xml.etree.ElementTree as ET

All tags:
{'link', 'plate_number_image_url', 'plate_id', 'plate_title', 'tags', 'plate_region', 'fon_id', 'fon_title', 'model', 'photo_url', 'plate_number', 'country', 'model2', 'car'}

In [42]:
car_pattern = r"([a-zA-Z0-9а-яА-Я]+)"

In [43]:
def get_folder_name(generation: str) -> str:
    """
    Generation name looks like a mess. This func will extract only numeric info
    :param generation:
    :return: number or string like "2nd"
    """
    if generation is None: return '0'

    match = re.search(car_pattern, generation)
    if match:
        return match.group(1)

    try:
        int(generation)
        return generation
    except ValueError:
        gen = generation.split()[0]
        if gen[-1] in ',:.;':
            gen = gen[:-1]
        return gen

In [44]:
def get_car_image_name(url: str) -> str:
    """
    Function to get unique name for umage. All url are unique (as long as pictures are different), so why don't we use that
    """
    name = url.split('/')[-1]
    # name = name.split('.')[0]
    return name

In [45]:
def remove_slash_and_other_trash(model:str) -> str:
    """
    We don't need lots of nested folders!
    """
    if model:
        return model.replace("/", "_&_").replace(":", "-")
    else:
        return '0'

In [46]:
def download_image(url, save_path):
    response = requests.get(url)
    response.raise_for_status()

    with open(save_path, 'wb') as file:
        file.write(response.content)

    print(f"Image downloaded at: {save_path}")

In [39]:
mypolicy = asyncio.WindowsSelectorEventLoopPolicy()
asyncio.set_event_loop_policy(mypolicy)

In [51]:
url = "https://platesmania.com/by/foto21956725"
response = requests.get(url)

In [53]:
response.status_code

403

In [38]:
url = "https://pixabay.com/api/?key=38179224-0b24f57dabb1085a721a641a5&id=21967248"
response = requests.get(url)
response.raise_for_status()
save_path = None
with open(save_path, 'wb') as file:
    file.write(response.content)

print(f"Image downloaded at: {save_path}")

HTTPError: 400 Client Error: Bad Request for url: https://pixabay.com/api/?key=38179224-0b24f57dabb1085a721a641a5&id=21967248

In [ ]:
url = "https://pixabay.com/api/"

body = {
    "model": "gpt-3.5-turbo",
    # "model": "gpt-4",

    "messages": [
        {"role": "system",
         "content": MODEL_INSTRUCTION[lang]},
        *data,
    ],

    "max_tokens": max_tokens,
    "temperature": temperature,
    "user": str(id),
    "n": n,
    "stream": False,
    "stop": None,
    # "stop": 'stop',
    "presence_penalty": 0,

}

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer {0}".format(GPT_TOKEN)  # Обращение к GPT через API-ключ
}

# Отправка асинхронных http-запросов
try:
    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=body) as response:
            answer = await response.json()
            if response.status == 200:
                # Получаем объект dict
                coins = answer['usage']['total_tokens']
                print(coins)
                return answer['choices'][0]['message']['content'], coins
            else:  # bad request - вызываем исключение
                try:
                    if answer['error']['code'] == 'context_length_exceeded':
                        return 'length_error',0
                    else:
                        raise Exception
                except:
                    raise Exception
except Exception as e:
    print(e)
    return '', 0


In [47]:
async def download_image_asynchronously(url, save_path):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            response.raise_for_status()
            with open(save_path, 'wb') as file:
                while True:
                    chunk = await response.content.read(1024)
                    if not chunk:
                        break
                    file.write(chunk)

    print(f"Image downloaded asynchronously at: {save_path}")


In [48]:
TEST_URL = "https://cdn.pixabay.com/photo/2023/07/01/18/56/dog-8100754_1280.jpg"

In [49]:
def parse_xml(xml_files_path: str, save_result_to: str, test_mode=True):
    """ Function to parse all XML files in folder and create all folders according to model names"""
    save_result_to += "/"
    xml_files = os.listdir(xml_files_path)
    for file_n, file_path in enumerate(xml_files[:2]):
        with open(xml_files_path + file_path, 'r', encoding='utf-8') as file:
            tree = ET.parse(file)
        root = tree.getroot()

        # Access elements and attributes in the XML file
        for n, child in enumerate(root[:2]):

            car = child.find("car").text
            if not car or len(car) < 2:
                continue
            photo_url = child.find("photo_url").text

            image_name = get_car_image_name(photo_url)

            url = TEST_URL if test_mode else photo_url


            model = child.find("model").text
            model2 = child.find("model2").text


            tags = [car, model, model2]

            for tag_n, tag in enumerate(tags[:], start=0):
                tag = get_folder_name(tag)
                tags[tag_n] = remove_slash_and_other_trash(tag)

            picture_path = save_result_to + '/'.join(tags)
            try:
                os.makedirs(picture_path, exist_ok=True)
                download_image(url=url, save_path=picture_path + '/' + image_name)
            except:
                raise
        print(f"File {file_n} is processed successfully.")


In [50]:
parse_xml(xml_files_path="../50k/", save_result_to="../XML_parsed_images/", test_mode=False)

HTTPError: 403 Client Error: Forbidden for url: https://img03.platesmania.com/230704/o/21967248.jpg

##### For Python sctipt we have to run loop to download asynchronously:

In [ ]:
# asyncio.run(download_image(url="", save_path=""))  # But it won't work in Jupyter

##### For Jupyter, we already have a loop, so we can just await the fucntion:

In [ ]:
# await download_image(url="", save_path="")